In [1]:
import sys
sys.path.insert(0, '../')

from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()

TTC = TimeTaggerController()
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(1)
TTC.set_alice_transmission_channel(2)
# set bob channels
TTC.set_bob_reflection_channel(4)
TTC.set_bob_transmission_channel(3)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()




ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [3]:
#_ = TTC.performDelayAdjustment(integration_time=5)#, manual_delays=[0, 947, 78, -502])

In [4]:
n_values = 150
trace_width = 3
#TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
#TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

In [5]:
# Global flag to control the loop
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, 11.5, -11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, 11.5), (0, -11.5), (-22.5, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

# for this perf we only want Q_all
angle_pairs = angle_pair_dict['Q_all']

# setup system with correct initial angles
KMC.rotate_simulataneously(*angle_pairs[0])

def performance_cycle(cycles):
    """The function you want to execute repeatedly."""
   
    time_of_last_visual = time.time()
    prev_theta_a, prev_theta_b = angle_pairs[0]
    for _ in range(cycles):
        theta_a, theta_b = random.choice(angle_pairs)
        
        measurement_results, prev_theta_a, prev_theta_b = TTC.get_single_measurement_metronome(angle_pairs, theta_a, theta_b, prev_theta_a, prev_theta_b, integration_time=0.05, max_time=0.1, min_event_count=1)
        

In [6]:
TTC.createCoincidenceChannels(0.5e-9)

In [205]:
start = time.perf_counter()
result = TTC.collect_stream_data_single_attempt(0.065, 0.065)
print(time.perf_counter() - start, result )

0.06521200000133831 [1002 1004 1004 1004 1004 1002 1004 1004 1001 1001 1003 1004 1004 1004
 1002 1001 1004 1004 1002 1002 1004 1004 1001 1004 1004 1001 1002 1001
 1004 1003 1001 1003 1003 1001 1002 1002 1001 1003 1001 1004 1004 1001
 1001 1004 1004 1004 1001 1003 1004 1001 1002 1001 1002 1004 1004 1003
 1001 1001 1003 1001 1001 1001 1004 1001 1001 1001 1004 1003 1004 1004
 1001 1004 1002 1001 1004 1004 1001 1004]


In [ ]:
performance_cycle(10)

Collected 213 events in 0.12 seconds.
Timing Summary: {'stream_loop': 0.1203823999967426, 'rotate_simultaneously': 0.30992200000036974, 'total': 0.430341800005408}
Collected 46 events in 0.06 seconds.
Timing Summary: {'stream_loop': 0.06287129998963792, 'rotate_simultaneously': 0.3090807999979006, 'total': 0.3720016000006581}
Collected 38 events in 0.06 seconds.
Timing Summary: {'stream_loop': 0.06264229999214876, 'rotate_simultaneously': 0.3413689000008162, 'total': 0.40440089999174234}
Collected 55 events in 0.06 seconds.
Timing Summary: {'rotate_simultaneously': 0.3109106000047177, 'stream_loop': 0.06493670000054408, 'total': 0.37587299999722745}
Collected 55 events in 0.06 seconds.
Timing Summary: {'rotate_simultaneously': 0.3044285000069067, 'stream_loop': 0.0636532999924384, 'total': 0.3681075999920722}
Collected 54 events in 0.06 seconds.
Timing Summary: {'rotate_simultaneously': 0.3091287000133889, 'stream_loop': 0.06500210000376683, 'total': 0.3741522000054829}
Collected 58 ev

# TODO

I will implement the double buffer and make sure they can achieve metronomic clicks by correcting the timing. I might need to pass timing data from previous to next function